In [3]:
import pandas as pd

file_path = 'Students_Grading_Dataset.csv'
data = pd.read_csv(file_path)

In [4]:
data

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,S5995,Ahmed,Jones,student4995@university.com,Male,19,Business,NaN,82.15,60.33,...,58.42,85.21,D,25.5,No,Yes,High School,Low,10,8.3
4996,S5996,Emma,Brown,student4996@university.com,Male,19,Business,65.11,86.31,49.80,...,60.87,95.96,C,5.0,No,Yes,NaN,Medium,4,4.0
4997,S5997,John,Brown,student4997@university.com,Female,24,CS,87.54,63.55,64.21,...,82.65,54.25,A,24.8,Yes,No,High School,Medium,4,6.3
4998,S5998,Sara,Davis,student4998@university.com,Male,23,CS,92.56,79.79,94.28,...,94.29,55.84,A,16.1,Yes,Yes,Bachelor's,Low,1,8.4


In [ ]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os

load_dotenv()

# Print columns to verify
print("Dataset columns:", data.columns.tolist())

# Convert string boolean values to integer
def to_boolean(value):
    if pd.isna(value):  # Handle NaN
        return 0
    if isinstance(value, str):
        value = value.lower()
        if value in ['yes', 'true', '1']:
            return 1
        if value in ['no', 'false', '0']:
            return 0
    return int(value) if value else 0

# Connect to Aiven MySQL
try:
    connection = mysql.connector.connect(
        host=os.getenv('mysql_host'),
        port=os.getenv('mysql_port'),
        database=os.getenv('mysql_database'),
        user=os.getenv('mysql_user'),
        password=os.getenv('mysql_password')
    )
    if connection.is_connected():
        print("Connected to Aiven MySQL")

        cursor = connection.cursor()

        # Insert data using stored procedure
        for index, row in data.iterrows():
            try:
                cursor.callproc('AddNewStudent', (
                    str(row.get('Student_ID', f'S{index:03d}')),  # Default ID if missing
                    row.get('First_Name', 'Unknown'),
                    row.get('Last_Name', 'Unknown'),
                    row.get('Email', None),
                    row.get('Gender', None),
                    row.get('Age', None),
                    row.get('Department', 'Unknown'),
                    row.get('Attendance_Percentage', 0),
                    row.get('Midterm_Score', 0),
                    row.get('Final_Score', 0),
                    row.get('Assignments_Avg', 0),
                    row.get('Quizzes_Avg', 0),
                    row.get('Participation_Score', 0),
                    row.get('Projects_Score', 0),
                    row.get('Total_Score', 0),
                    row.get('Grade', 'N/A'),
                    row.get('Study_Hours_per_Week', 0),
                    row.get('Sleep_Hours_per_Night', 0),
                    row.get('Stress_Level', 0),
                    to_boolean(row.get('Extracurricular_Activities', 0)),
                    to_boolean(row.get('Internet_Access_at_Home', 0)),
                    row.get('Parent_Education_Level', 'Unknown'),
                    row.get('Family_Income_Level', 'Unknown')
                ))
                connection.commit()
                print(f"Inserted row {index} for Student_ID: {row.get('Student_ID', f'S{index:03d}')}")
            except Error as e:
                print(f"Error inserting row {index}: {e}")

        print("Data loaded successfully")

except Error as e:
    print(f"Connection error: {e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection closed")

Dataset columns: ['Student_ID', 'First_Name', 'Last_Name', 'Email', 'Gender', 'Age', 'Department', 'Attendance (%)', 'Midterm_Score', 'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score', 'Projects_Score', 'Total_Score', 'Grade', 'Study_Hours_per_Week', 'Extracurricular_Activities', 'Internet_Access_at_Home', 'Parent_Education_Level', 'Family_Income_Level', 'Stress_Level (1-10)', 'Sleep_Hours_per_Night']
Connected to Aiven MySQL
Error inserting row 0: 1406 (22001): Data too long for column 'p_Gender' at row 1
Error inserting row 1: 1054 (42S22): Unknown column 'nan' in 'field list'
Error inserting row 2: 1406 (22001): Data too long for column 'p_Gender' at row 1
Error inserting row 3: 1406 (22001): Data too long for column 'p_Gender' at row 1
Error inserting row 4: 1406 (22001): Data too long for column 'p_Gender' at row 1
Error inserting row 5: 1406 (22001): Data too long for column 'p_Gender' at row 1
Error inserting row 6: 1054 (42S22): Unknown column 'nan' in 'fie

In [ ]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv

load_dotenv()


client = MongoClient(os.getenv('mongo_db_url'))

db = client["ml_dataset_design"]

# Collection name
collections = {
    'students': db['students'],
    'academic_details': db['academic_details'],
    'study_habits': db['study_habits'],
    'family_background': db['family_background'],
    'extracurriculars': db['extracurriculars']
}

# To convert string to boolean
def to_boolean(value):
    if pd.isna(value):
        return False
    if isinstance(value, str):
        value = value.lower()
        if value in ['yes', 'true', 'y', '1']:
            return True
        if value in ['no', 'false', 'n', '0']:
            return False
    return bool(value)

# Split data into collections
student_data = []
academic_data = []
study_habits_data = []
extracurricular_data = []
family_background_data = []

for index, row in data.iterrows():
    student_id = str(row.get('Student_ID', f'S{index:03d}'))  # Default ID if missing

    # Students collection
    student_data.append({
        'Student_ID': student_id,
        'First_Name': row.get('First_Name', 'Unknown'),
        'Last_Name': row.get('Last_Name', 'Unknown'),
        'Email': row.get('Email', None),
        'Gender': row.get('Gender', None),
        'Age': row.get('Age', None)
    })

    # Academic_Details collection
    academic_data.append({
        'Student_ID': student_id,
        'Department': row.get('Department', 'Unknown'),
        'Attendance_Percentage': row.get('Attendance_Percentage', 0),
        'Midterm_Score': row.get('Midterm_Score', 0),
        'Final_Score': row.get('Final_Score', 0),
        'Assignments_Avg': row.get('Assignments_Avg', 0),
        'Quizzes_Avg': row.get('Quizzes_Avg', 0),
        'Participation_Score': row.get('Participation_Score', 0),
        'Projects_Score': row.get('Projects_Score', 0),
        'Total_Score': row.get('Total_Score', 0),
        'Grade': row.get('Grade', 'N/A')
    })

    # Study_Habits collection
    study_habits_data.append({
        'Student_ID': student_id,
        'Study_Hours_per_Week': row.get('Study_Hours_per_Week', 0),
        'Sleep_Hours_per_Night': row.get('Sleep_Hours_per_Night', 0),
        'Stress_Level': row.get('Stress_Level', 0)
    })

    # Extracurriculars collection
    extracurricular_data.append({
        'Student_ID': student_id,
        'Extracurricular_Activities': to_boolean(row.get('Extracurricular_Activities', False))
    })

    # Family_Background collection
    family_background_data.append({
        'Student_ID': student_id,
        'Internet_Access_at_Home': to_boolean(row.get('Internet_Access_at_Home', False)),
        'Parent_Education_Level': row.get('Parent_Education_Level', 'Unknown'),
        'Family_Income_Level': row.get('Family_Income_Level', 'Unknown')
    })

# Insert data into each collection
collections['students'].insert_many(student_data)
print(f"Inserted {len(student_data)} documents into students")
collections['academic_details'].insert_many(academic_data)
print(f"Inserted {len(academic_data)} documents into academic_details")
collections['study_habits'].insert_many(study_habits_data)
print(f"Inserted {len(study_habits_data)} documents into study_habits")
collections['extracurriculars'].insert_many(extracurricular_data)
print(f"Inserted {len(extracurricular_data)} documents into extracurriculars")
collections['family_background'].insert_many(family_background_data)
print(f"Inserted {len(family_background_data)} documents into family_background")

# Verify sample data
for coll_name, coll in collections.items():
    print(f"\nSample from {coll_name}:")
    sample_docs = coll.find().limit(2)
    for doc in sample_docs:
        print(doc)

# Close connection
client.close()

Inserted 5000 documents into students
Inserted 5000 documents into academic_details
Inserted 5000 documents into study_habits
Inserted 5000 documents into extracurriculars
Inserted 5000 documents into family_background

Sample from students:
{'_id': ObjectId('67cd24d994a02154b2e19ef7'), 'Student_ID': 'S1000', 'First_Name': 'Omar', 'Last_Name': 'Williams', 'Email': 'student0@university.com', 'Gender': 'Female', 'Age': 22}
{'_id': ObjectId('67cd24d994a02154b2e19ef8'), 'Student_ID': 'S1001', 'First_Name': 'Maria', 'Last_Name': 'Brown', 'Email': 'student1@university.com', 'Gender': 'Male', 'Age': 18}

Sample from academic_details:
{'_id': ObjectId('67cd24e094a02154b2e1b27f'), 'Student_ID': 'S1000', 'Department': 'Engineering', 'Attendance_Percentage': 0, 'Midterm_Score': 55.03, 'Final_Score': 57.82, 'Assignments_Avg': 84.22, 'Quizzes_Avg': 74.06, 'Participation_Score': 3.99, 'Projects_Score': 85.9, 'Total_Score': 56.09, 'Grade': 'F'}
{'_id': ObjectId('67cd24e094a02154b2e1b280'), 'Student_I